In [ ]:
!pip install keras-tcn --quiet


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving BAJFINANCE.csv to BAJFINANCE.csv
Saving BHARTIARTL.csv to BHARTIARTL.csv
Saving HDFCBANK.csv to HDFCBANK.csv
Saving INFY.csv to INFY.csv
Saving ITC.csv to ITC.csv
Saving LT.csv to LT.csv
Saving RELIANCE.csv to RELIANCE.csv
Saving SUNPHARMA.csv to SUNPHARMA.csv
Saving TATACHEM.csv to TATACHEM.csv
Saving TATAMOTORS.csv to TATAMOTORS.csv


In [ ]:
!pip install ta --quiet

  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")

stock_files = {
    'HDFC Bank': 'HDFCBANK.csv',
    'Reliance': 'RELIANCE.csv',
    'Infosys': 'INFY.csv',
    'ITC': 'ITC.csv',
    'Tata Motors': 'TATAMOTORS.csv',
    'Sun Pharma': 'SUNPHARMA.csv',
    'L&T': 'LT.csv',
    'Bharti Airtel': 'BHARTIARTL.csv',
    'Bajaj Finance': 'BAJFINANCE.csv',
    'Tata Chemicals': 'TATACHEM.csv'
}

results = []

for stock_name, file_path in stock_files.items():
    try:
        df = pd.read_csv(file_path)
        df.columns = [col.lower() for col in df.columns]
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df.set_index('date', inplace=True)
        df['close'] = pd.to_numeric(df['close'], errors='coerce')
        df.dropna(subset=['close'], inplace=True)

        # Add indicators
        df['rsi'] = RSIIndicator(close=df['close'], window=14).rsi()
        df['macd'] = MACD(close=df['close']).macd()
        bb = BollingerBands(close=df['close'])
        df['bb_upper'] = bb.bollinger_hband()
        df['bb_lower'] = bb.bollinger_lband()
        df['bb_mavg'] = bb.bollinger_mavg()

        # Create target
        df['target'] = (df['close'].shift(-1) > df['close']).astype(int)
        df.dropna(inplace=True)

        # Features
        features = ['close', 'rsi', 'macd', 'bb_upper', 'bb_lower', 'bb_mavg']
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(df[features])

        # Sequence data
        window = 60
        X, y = [], []
        for i in range(window, len(df)):
            X.append(scaled[i-window:i])
            y.append(df['target'].iloc[i])
        X, y = np.array(X), np.array(y)

        # Split
        split = int(0.8 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        # Build TCN model
        model = Sequential()
        model.add(TCN(input_shape=(X_train.shape[1], X_train.shape[2]), nb_filters=64, kernel_size=3))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        model.fit(X_train, y_train, epochs=25, batch_size=32, validation_split=0.1, callbacks=[es], verbose=0)

        # Predict
        y_proba = model.predict(X_test).flatten()
        y_pred = (y_proba > 0.5).astype(int)

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred)
        rec = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)

        # Strategy performance
        test_df = df.iloc[-len(y_test):].copy()
        test_df['pred'] = y_pred
        test_df['return'] = test_df['close'].pct_change()
        test_df['position'] = test_df['pred'].replace({0: -1, 1: 1})
        test_df['strategy'] = test_df['position'].shift(1) * test_df['return']
        test_df.dropna(inplace=True)

        sharpe = (test_df['strategy'].mean() / test_df['strategy'].std()) * np.sqrt(252) if test_df['strategy'].std() != 0 else 0
        cum = (1 + test_df['strategy']).cumprod()
        dd = (cum.cummax() - cum) / cum.cummax()
        max_dd = dd.max()

        results.append({
            'Stock': stock_name,
            'Accuracy': round(acc, 2),
            'Precision': round(prec, 2),
            'Recall': round(rec, 2),
            'F1 Score': round(f1, 2),
            'ROC AUC': round(roc_auc, 2),
            'Sharpe Ratio': round(sharpe, 2),
            'Max Drawdown (%)': round(max_dd * 100, 2)
        })

        print(f"✅ {stock_name} TCN done")

    except Exception as e:
        print(f"❌ {stock_name} failed: {e}")


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
✅ HDFC Bank TCN done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
✅ Reliance TCN done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
✅ Infosys TCN done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step
✅ ITC TCN done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
✅ Tata Motors TCN done
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step
✅ Sun Pharma TCN done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
✅ L&T TCN done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
✅ Bharti Airtel TCN done
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step
✅ Bajaj Finance TCN done
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
✅ Tata Chemicals TCN done


In [ ]:
results_df = pd.DataFrame(results)
results_df


,Stock,Accuracy,Precision,Recall,F1 Score,ROC AUC,Sharpe Ratio,Max Drawdown (%)
0,HDFC Bank,0.53,0.53,0.92,0.68,0.49,0.34,19.03
1,Reliance,0.51,0.52,0.45,0.48,0.50,0.01,29.40
2,Infosys,0.50,0.50,0.63,0.56,0.52,0.24,29.27
3,ITC,0.48,0.49,0.92,0.64,0.45,1.05,16.84
4,Tata Motors,0.50,0.49,0.36,0.42,0.49,-0.43,49.54
5,Sun Pharma,0.54,0.54,1.00,0.70,0.54,1.86,12.24
6,L&T,0.53,0.54,0.76,0.63,0.50,-0.06,26.98
7,Bharti Airtel,0.50,0.51,0.71,0.60,0.53,0.63,24.35
8,Bajaj Finance,0.50,0.49,0.62,0.55,0.50,-0.31,35.24
9,Tata Chemicals,0.51,0.52,0.82,0.64,0.50,-0.29,36.71


In [ ]:
# Save LSTM results
lstm_df = pd.DataFrame(results)  # Assuming your LSTM results are in `results`
lstm_df.to_csv('tcn_results.csv', index=False)

# Download
from google.colab import files
files.download('tcn_results.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>